In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
z=zipfile.ZipFile('../input/facial-keypoints-detection/training.zip','r')
z.extractall(path=r"./")
z.close()

In [ ]:
z=zipfile.ZipFile('../input/facial-keypoints-detection/test.zip','r')
z.extractall(path=r"./")
z.close()

In [ ]:
train_data=pd.read_csv('./training.csv')
test_data=pd.read_csv('./test.csv')
lookid_data = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv')

In [ ]:
import matplotlib.pyplot as plt

imag = []
for i in range(0,7049):
    img = train_data['Image'][i].split(' ')
    img = ['0' if x == '' else x for x in img]
    imag.append(img)
    
image_list = np.array(imag,dtype = 'float')
X_train = image_list.reshape(-1,96,96,1)

In [ ]:
train_data.fillna(method = 'ffill',inplace = True)

In [ ]:
training = train_data.drop('Image',axis = 1)

y_train = []
for i in range(0,7049):
    y = training.iloc[i,:]

    y_train.append(y)
y_train = np.array(y_train,dtype = 'float')

In [ ]:
model = Sequential()
model.add(Conv2D(24,kernel_size=3,padding='same',activation='relu',input_shape=(96,96,1)))
model.add(BatchNormalization())
model.add(Conv2D(24,kernel_size=3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(24,kernel_size=5,strides=2,padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(48,kernel_size=3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(48,kernel_size=3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(48,kernel_size=5,strides=2,padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(60,kernel_size=3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(60,kernel_size=3,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(60,kernel_size=5,strides=2,padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
# CREATE VALIDATION SET
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, y_train, test_size = 0.333)
# TRAIN NETWORKS
epochs = 100
history= model.fit(X_train2,Y_train2, batch_size=80, epochs = epochs, validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=1)

In [ ]:
timag = []
for i in range(0,1783):
    timg = test_data['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]
    
    timag.append(timg)

In [ ]:
timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1) 

In [ ]:
pred = model.predict(X_test)

In [ ]:
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)

In [ ]:
rowid = lookid_data['RowId']
rowid=list(rowid)

In [ ]:
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))

In [ ]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])

In [ ]:
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('submission.csv',index = False)